In [1]:
pip install snowflake-connector-python


  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached boto3-1.40.22-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.22-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.9 MB/s eta 0:00:00
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
Using cached boto3-1.40.22-py3-none-any.whl (139 kB)
Using cached botocore-1.40.22-py3-none-any.whl (14.0 MB)
Using cached s3transfer-0.13.1-py3-none-any.whl (85 kB)

   ---------------------------------------- 0/5 [asn1crypto]
  Attempting uninstall: botocore
   ---------------------------------------- 0/5 [asn1crypto]
   -------- ------------------------------- 1/5 [botocore]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.40.22 which is incompatible.


In [ ]:
#Query a SnowFlake Table
import snowflake.connector


# Establish Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

cur = conn.cursor()

try:
    # Step 1: Query the total number of rows in the table
    print("📊 Querying the total number of rows in CREDITCARD.PUBLIC.CREDITCARD...")
    cur.execute("SELECT COUNT(*) FROM CREDITCARD.PUBLIC.CREDITCARD;")
    
    # Fetch the result
    result = cur.fetchone()
    
    # Extract the row count (the result is a tuple, e.g., (row_count,))
    row_count = result[0]
    print(f"✅ Total number of rows: {row_count}")

    # Step 2: Query the top 10 rows from the table
    print("🔍 Querying the top 10 rows from CREDITCARD.PUBLIC.CREDITCARD...")
    cur.execute("SELECT * FROM CREDITCARD.PUBLIC.CREDITCARD LIMIT 10;")
    
    # Fetch the top 10 rows
    top_rows = cur.fetchall()
    
    # Print the top 10 rows in a clean format
    print("✅ Top 10 rows:")
    
    # Fetch column names
    columns = [desc[0] for desc in cur.description]  # Get column names from cursor description

    # Print column headers
    print(f"{' | '.join(columns)}")  # Joining column names with ' | '

    # Print each row in a clean format
    for row in top_rows:
        print(f"{' | '.join(map(str, row))}")  # Join each element in the row as a string and print

finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()
    print("🔒 Connection closed.")


📊 Querying the total number of rows in CREDITCARD.PUBLIC.CREDITCARD...
✅ Total number of rows: 149999
🔍 Querying the top 10 rows from CREDITCARD.PUBLIC.CREDITCARD...
✅ Top 10 rows:
TIME | V1 | V2 | V3 | V4 | V5 | V6 | V7 | V8 | V9 | V10 | V11 | V12 | V13 | V14 | V15 | V16 | V17 | V18 | V19 | V20 | V21 | V22 | V23 | V24 | V25 | V26 | V27 | V28 | AMOUNT | CLASS
0 | -1.359807134 | -0.072781173 | 2.536346738 | 1.378155224 | -0.33832077 | 0.462387778 | 0.239598554 | 0.098697901 | 0.36378697 | 0.090794172 | -0.551599533 | -0.617800856 | -0.991389847 | -0.311169354 | 1.468176972 | -0.470400525 | 0.207971242 | 0.02579058 | 0.40399296 | 0.251412098 | -0.018306778 | 0.277837576 | -0.11047391 | 0.066928075 | 0.128539358 | -0.189114844 | 0.133558377 | -0.021053053 | 149.62 | 0
0 | 1.191857111 | 0.266150712 | 0.166480113 | 0.448154078 | 0.060017649 | -0.082360809 | -0.078802983 | 0.085101655 | -0.255425128 | -0.166974414 | 1.612726661 | 1.065235311 | 0.489095016 | -0.143772296 | 0.635558093 | 0.463

In [ ]:
#Upload Fresh File and Replace Previous
import snowflake.connector
import csv


# Establish Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,  # Updated account URL part
    warehouse=warehouse,
    database=database,
    schema=schema
)



# Table components
database = 'CREDITCARD'
schema = 'PUBLIC'
table = 'CREDITCARD'

csv_file_path = r'C:\Users\sajag\Desktop\Data_Split\Training.csv'

try:
    cur = conn.cursor()

    # Set the database and schema context
    cur.execute(f"USE DATABASE {database}")
    cur.execute(f"USE SCHEMA {schema}")

    print(f"🔄 Truncating table: {table}")
    cur.execute(f"TRUNCATE TABLE {table};")

    # Step 1: Upload file to internal stage of the table
    print("📤 Uploading CSV file to internal Snowflake stage...")
    cur.execute(f"PUT file://{csv_file_path} @%{table} OVERWRITE = TRUE")

    # Step 2: Bulk load data using COPY INTO
    print("📥 Loading data into Snowflake table...")
    copy_sql = f"""
        COPY INTO {table}
        FROM @%{table}/Training.csv
        FILE_FORMAT = (
            TYPE = 'CSV'
            FIELD_OPTIONALLY_ENCLOSED_BY = '"'
            SKIP_HEADER = 1
        );
    """
    cur.execute(copy_sql)

    print("✅ Data successfully uploaded and loaded to Snowflake.")

finally:
    cur.close()
    conn.close()
    print("🔒 Connection closed.")


🔄 Truncating table: CREDITCARD
📤 Uploading CSV file to internal Snowflake stage...
📥 Loading data into Snowflake table...
✅ Data successfully uploaded and loaded to Snowflake.
🔒 Connection closed.


In [ ]:
# Append New File
import snowflake.connector
import csv

# Snowflake credentials

       # Your schema (replace if different)

# Establish Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,  # Updated account URL part
    warehouse=warehouse,
    database=database,
    schema=schema
)



# Table components
database = 'CREDITCARD'
schema = 'PUBLIC'
table = 'CREDITCARD'

csv_file_path = r'C:\Users\sajag\Desktop\Data_Split\Append_1.csv'

try:
    cur = conn.cursor()

    # Set the database and schema context
    cur.execute(f"USE DATABASE {database}")
    cur.execute(f"USE SCHEMA {schema}")


    # Step 1: Upload file to internal stage of the table
    print("📤 Uploading CSV file to internal Snowflake stage...")
    cur.execute(f"PUT file://{csv_file_path} @%{table} OVERWRITE = TRUE")

    # Step 2: Bulk load data using COPY INTO
    print("📥 Loading data into Snowflake table...")
    copy_sql = f"""
        COPY INTO {table}
        FROM @%{table}/Append_1.csv
        FILE_FORMAT = (
            TYPE = 'CSV'
            FIELD_OPTIONALLY_ENCLOSED_BY = '"'
            SKIP_HEADER = 1
        );
    """
    cur.execute(copy_sql)

    print("✅ Data successfully uploaded and loaded to Snowflake.")

finally:
    cur.close()
    conn.close()
    print("🔒 Connection closed.")


📤 Uploading CSV file to internal Snowflake stage...
📥 Loading data into Snowflake table...
✅ Data successfully uploaded and loaded to Snowflake.
🔒 Connection closed.
